In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
batch_size = 128
epochs = 10
seed = 1
log_interval=10

In [3]:
device = torch.device("cpu")

kwargs = {} #{'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [4]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc2 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc2(h1)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        z = self.encode(x.view(-1, 784))
        return self.decode(z)


model = AE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
def vae_loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [7]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = vae_loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [8]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += vae_loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
if __name__ == "__main__":
    for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 551.264282
Train Epoch: 1 [1280/60000 (2%)]	Loss: 289.760529
Train Epoch: 1 [2560/60000 (4%)]	Loss: 237.383118
Train Epoch: 1 [3840/60000 (6%)]	Loss: 220.868011
Train Epoch: 1 [5120/60000 (9%)]	Loss: 215.439819
Train Epoch: 1 [6400/60000 (11%)]	Loss: 212.726212
Train Epoch: 1 [7680/60000 (13%)]	Loss: 209.082657
Train Epoch: 1 [8960/60000 (15%)]	Loss: 203.099472
Train Epoch: 1 [10240/60000 (17%)]	Loss: 189.897797
Train Epoch: 1 [11520/60000 (19%)]	Loss: 188.201050
Train Epoch: 1 [12800/60000 (21%)]	Loss: 179.652267
Train Epoch: 1 [14080/60000 (23%)]	Loss: 173.603058
Train Epoch: 1 [15360/60000 (26%)]	Loss: 163.888901
Train Epoch: 1 [16640/60000 (28%)]	Loss: 165.033752
Train Epoch: 1 [17920/60000 (30%)]	Loss: 162.400177
Train Epoch: 1 [19200/60000 (32%)]	Loss: 157.503952
Train Epoch: 1 [20480/60000 (34%)]	Loss: 164.107056
Train Epoch: 1 [21760/60000 (36%)]	Loss: 146.162567
Train Epoch: 1 [23040/60000 (38%)]	Loss: 158.221405
Train Epoch: 1 [24320/60000 

Train Epoch: 4 [17920/60000 (30%)]	Loss: 110.194138
Train Epoch: 4 [19200/60000 (32%)]	Loss: 108.052681
Train Epoch: 4 [20480/60000 (34%)]	Loss: 108.570282
Train Epoch: 4 [21760/60000 (36%)]	Loss: 114.364189
Train Epoch: 4 [23040/60000 (38%)]	Loss: 113.817108
Train Epoch: 4 [24320/60000 (41%)]	Loss: 110.218437
Train Epoch: 4 [25600/60000 (43%)]	Loss: 107.343163
Train Epoch: 4 [26880/60000 (45%)]	Loss: 112.080383
Train Epoch: 4 [28160/60000 (47%)]	Loss: 111.943863
Train Epoch: 4 [29440/60000 (49%)]	Loss: 108.065193
Train Epoch: 4 [30720/60000 (51%)]	Loss: 111.912086
Train Epoch: 4 [32000/60000 (53%)]	Loss: 111.785767
Train Epoch: 4 [33280/60000 (55%)]	Loss: 111.056778
Train Epoch: 4 [34560/60000 (58%)]	Loss: 115.110168
Train Epoch: 4 [35840/60000 (60%)]	Loss: 111.672775
Train Epoch: 4 [37120/60000 (62%)]	Loss: 107.917725
Train Epoch: 4 [38400/60000 (64%)]	Loss: 110.659439
Train Epoch: 4 [39680/60000 (66%)]	Loss: 114.561127
Train Epoch: 4 [40960/60000 (68%)]	Loss: 111.906204
Train Epoch:

In [ ]:
sample = torch.randn(64, 20).to(device)
sample = model.decode(sample).cpu()


In [ ]:
s=sample.view(64,1,28,28)

In [ ]:
k=s[0].detach().numpy()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(k[0])